In [27]:
import wordpiece as wp
import pandas as pd
import nltk
from string import punctuation, ascii_lowercase
from collections import defaultdict

In [28]:
text = """
prepara la pasta frolla mescolando 300g di farina 150g di burro 100g di zucchero 1 uovo e un pizzico di sale 
impasta fino a ottenere un composto omogeneo avvolgi in pellicola e lascia riposare in frigo per 30 minuti  
stendi la pasta frolla su una superficie infarinata e rivesti uno stampo da crostata imburrato  
bucherella il fondo con una forchetta e cuoci in forno preriscaldato a 180°c per 20 minuti  
prepara la crema pasticcera mescolando 500ml di latte 4 tuorli 100g di zucchero 40g di farina e una scorza di limone  
cuoci a fuoco lento fino a ottenere una crema densa poi lasciala raffreddare  versa la crema pasticcera 
sulla base della crostata raffreddata  decora con frutta fresca a piacere fragole kiwi mirtilli ecc  
scalda un cucchiaio di gelatina di albicocche e spennella la frutta per lucidarla  
lascia raffreddare in frigo per almeno 1 ora prima di servire
"""
print(len(text.split()))

147


In [37]:
example = "tutto fatto"

wt = wp.WordPieceTokenizer(text=text, maximum_size=200)

In [38]:
example = wt.score_pairs(normalize=False)
print(example.head()) 

#r  #e    18
#e  #r    16
#t  #a    13
#l  #a    12
d   #i    11
dtype: int64


In [39]:
wt.train(normalize=False)

 75%|███████▌  | 150/200 [00:00<00:00, 558.99it/s]


In [40]:
tokenized = []
for tokens in wt.corpus:
    tokenized.extend(list(tokens))
print(" | ".join(tokenized))

prepara | la | pasta | frolla | mescolando | 300g | di | farina | 150g | di | burro | 100g | di | zucchero | 1 | uovo | e | un | pizzico | di | sale | impasta | fino | a | ottenere | un | composto | omogeneo | avvolgi | in | pellicola | e | lascia | riposare | in | frigo | per | 3 | #0 | minuti | s | #ten | #d | #i | la | pasta | frolla | su | una | su | #p | #er | #f | #i | #ci | #e | in | #f | #arina | #ta | e | ri | #v | #es | #ti | un | #o | s | #ta | #mp | #o | d | #a | crostata | i | #m | #b | #u | #r | #ra | #to | bu | #ch | #ere | #lla | i | #l | f | #on | #do | con | una | for | #ch | #e | #tta | e | cuoci | in | for | #n | #o | pre | #ri | #sc | #alda | #to | a | 1 | #8 | #0 | #° | #c | per | 2 | #0 | minuti | prepara | la | crema | pasticcera | mescolando | 5 | #0 | #0 | #m | #l | di | la | #t | #t | #e | 4 | t | #uo | #r | #li | 100g | di | zucchero | 4 | #0g | di | farina | e | una | s | #co | #r | #z | #a | di | l | #i | #m | #on | #e | cuoci | a | f | #uo | #co | l | #en

In [41]:
V = wt.fancy_vocabulary
V.sort_values(ascending=False)

#a          88
#e          58
#r          57
#i          56
#o          51
            ..
pi           1
piz          1
pizz         1
pizzico      1
riposare     1
Length: 199, dtype: int64

In [43]:
test = "per preparare tutta la pasta frolla mescola 150g di farina"
print(wt.tokenize_text(test))

['per', 'prepara', '#re', 't', '#utta', 'la', 'pasta', 'frolla', 'mescola', '150g', 'di', 'farina']


In [44]:
test = "per eccellere nello sport serve molto allenamento"
print(wt.tokenize_text(test))

['per', 'e', '#cc', '#e', '#l', '#l', '#ere', '[UNK]', 's', '#p', '#or', '#t', 's', '#er', '#v', '#e', 'm', '#o', '#l', '#to', 'al', '#l', '#en', '#a', '#m', '#en', '#to']


## Process explained

In [45]:
text = """
prepara la pasta frolla mescolando 300g di farina 150g di burro 100g di zucchero 1 uovo e un pizzico di sale 
impasta fino a ottenere un composto omogeneo avvolgi in pellicola e lascia riposare in frigo per 30 minuti  
stendi la pasta frolla su una superficie infarinata e rivesti uno stampo da crostata imburrato  
bucherella il fondo con una forchetta e cuoci in forno preriscaldato a 180°c per 20 minuti  
prepara la crema pasticcera mescolando 500ml di latte 4 tuorli 100g di zucchero 40g di farina e una scorza di limone  
cuoci a fuoco lento fino a ottenere una crema densa poi lasciala raffreddare  versa la crema pasticcera 
sulla base della crostata raffreddata  decora con frutta fresca a piacere fragole kiwi mirtilli ecc  
scalda un cucchiaio di gelatina di albicocche e spennella la frutta per lucidarla  
lascia raffreddare in frigo per almeno 1 ora prima di servire
"""
example = "ottenere un"
current = ""
normalize = False 
wt = wp.WordPieceTokenizer(text=text, maximum_size=500)
iterations = list(range(wt.max_vocabulary_size))
for i in iterations:
    check = wt.update(normalize=normalize)
    tokens = wt.tokenize_text(example)
    if tokens != current:
        print("Iterazione {}: {}".format(i, " ".join(tokens)))
        current = tokens
    if not check:
        break 

Iterazione 0: o #t #t #e #n #e #re u #n
Iterazione 3: o #t #t #e #n #er #e u #n
Iterazione 6: o #t #t #en #er #e u #n
Iterazione 8: o #t #t #en #er #e un
Iterazione 20: o #t #t #en #ere un
Iterazione 39: o #t #ten #ere un
Iterazione 89: o #t #tenere un
Iterazione 91: o #ttenere un
Iterazione 107: ottenere un


In [29]:
len(wt.vocabulary)

335

In [30]:
for w, s in wt.fancy_vocabulary.items():
    if w.startswith('o'):
        print(w, s)

o 4
ottenere 2
om 1
omo 1
omog 1
omogen 1
omogene 1
omogeneo 1
ora 1


In [46]:
from transformers import BertTokenizer

In [48]:
tokenizer = BertTokenizer.from_pretrained("dbmdz/bert-base-italian-uncased")

## BERT Tokenizer

In [49]:
example = "mescolando mescolare mescolavi mescolai mescoli"

tokens = tokenizer.tokenize(example)
token_ids = tokenizer.convert_tokens_to_ids(tokens)

print("Testo originale:", example)
print("Token:", tokens)
print("Token IDs:", token_ids)

Testo originale: mescolando mescolare mescolavi mescolai mescoli
Token: ['mesco', '##lando', 'mescolare', 'mesco', '##lav', '##i', 'mesco', '##lai', 'mesco', '##li']
Token IDs: [10689, 2344, 29182, 10689, 19075, 30934, 10689, 16362, 10689, 125]
